In [30]:
import pyodbc
import pandas as pd

# Define your connection string
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=NGUYEN;"
    "DATABASE=CompanyX;"
    "Trusted_Connection=yes;"
    # "UID=YourUsername;"
    # "PWD=YourPassword;"
)

# Establish connection
conn = pyodbc.connect(conn_str)


In [31]:
# Write SQL query
query1 = "SELECT * FROM DimCustomer"

# Execute query and load data into a DataFrame
DimCustomer = pd.read_sql(query1, conn)

# Write SQL query
query2 = "SELECT * FROM DimSalesTerritory"

# Execute query and load data into a DataFrame
DimSalesTerritory = pd.read_sql(query2, conn)

# Write SQL query
query3 = "SELECT * FROM DimProduct"

# Execute query and load data into a DataFrame
DimProduct = pd.read_sql(query3, conn)

# Write SQL query
query4 = "SELECT * FROM DimDate"

# Execute query and load data into a DataFrame
DimDate = pd.read_sql(query4, conn)

# Write SQL query
query5 = "SELECT * FROM DimGeography"

# Execute query and load data into a DataFrame
DimGeography = pd.read_sql(query5, conn)

# Write SQL query
query6 = "SELECT * FROM DimPromotion"

# Execute query and load data into a DataFrame
DimPromotion = pd.read_sql(query6, conn)

# Write SQL query
query7 = "SELECT * FROM FactSales"

# Execute query and load data into a DataFrame
FactSales = pd.read_sql(query7, conn)


C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_27644\2241041406.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DimCustomer = pd.read_sql(query1, conn)
C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_27644\2241041406.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DimSalesTerritory = pd.read_sql(query2, conn)
C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_27644\2241041406.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DimProduct = pd.read_sql(query3, conn)
C:\Users\Khoi Nguyen\AppData\Local\Temp\ipyke

In [32]:
FactSales = pd.merge(FactSales, DimProduct[['ProductKey', 'ProductCategoryName']],
                     left_on='ProductID',
                     right_on='ProductKey',
                     how='left')

FactSales.drop(columns='ProductKey', inplace=True)

### Total Revenue per day per sales territory

In [33]:
sales_per_day_per_territory = (
    FactSales.groupby(["OrderDate"])["SalesAmount"]
    .sum()
    .reset_index())

# sales_per_day_per_territory = sales_per_day_per_territory.sort_values(by="OrderDate")

sales_per_day_per_territory.head()

,OrderDate,SalesAmount
0,2011-05-31,442160.4116
1,2011-06-01,13931.5200
2,2011-06-02,15012.1782
3,2011-06-03,7156.5400
4,2011-06-04,15012.1782


### Average Profit Margin per day per sales territory

In [34]:
FactSales['Average Profit Margin'] = ((FactSales['SalesAmount'] - FactSales['TotalProductCost']) / FactSales['TotalProductCost']) * 100
average_profit_per_day_per_territory = (
    FactSales.groupby(["OrderDate" ])[["Average Profit Margin", "OrderQty", "UnitPrice", "UnitPriceDiscount", "TaxAmt", "Freight"]]
    .sum()
    .reset_index()
)
average_profit_per_day_per_territory.head()

,OrderDate,Average Profit Margin,OrderQty,UnitPrice,UnitPriceDiscount,TaxAmt,Freight
0,2011-05-31,-4413.435852,735.0,211428.8337,0.0,35372.832928,11054.010263
1,2011-06-01,285.216668,4.0,13931.5200,0.0,1114.521600,348.288000
2,2011-06-02,302.834321,5.0,15012.1782,0.0,1200.974256,375.304455
3,2011-06-03,129.597896,2.0,7156.5400,0.0,572.523200,178.913500
4,2011-06-04,302.834321,5.0,15012.1782,0.0,1200.974256,375.304455


### Average revenue of All Customer per Sales Territory

In [35]:
result = (
    FactSales.groupby(["OrderDate"])
    .agg(
        TotalSalesAmount=("SalesAmount", "sum"),
        TotalCustomers=("CustomerID", "nunique")
    )
    .reset_index()
)

# Calculate the ratio: SalesAmount / Total Customers
result["Average_Revenue_Per_User"] = result["TotalSalesAmount"] / result["TotalCustomers"]

columns_to_drop = ["TotalSalesAmount", "TotalCustomers"]
result = result.drop(columns_to_drop, axis=1)

result.head()

,OrderDate,Average_Revenue_Per_User
0,2011-05-31,10784.400283
1,2011-06-01,3482.880000
2,2011-06-02,3002.435640
3,2011-06-03,3578.270000
4,2011-06-04,3002.435640


In [36]:
merged_data = pd.merge(
    sales_per_day_per_territory,
    average_profit_per_day_per_territory,
    on=["OrderDate"],
    how="inner"
)

merged_data = pd.merge(
    merged_data,
    result,
    on=["OrderDate"],
    how="inner"
)


In [38]:
# Display the final merged DataFrame
merged_data.head()

,OrderDate,SalesAmount,Average Profit Margin,OrderQty,UnitPrice,UnitPriceDiscount,TaxAmt,Freight,Average_Revenue_Per_User
0,2011-05-31,442160.4116,-4413.435852,735.0,211428.8337,0.0,35372.832928,11054.010263,10784.400283
1,2011-06-01,13931.5200,285.216668,4.0,13931.5200,0.0,1114.521600,348.288000,3482.880000
2,2011-06-02,15012.1782,302.834321,5.0,15012.1782,0.0,1200.974256,375.304455,3002.435640
3,2011-06-03,7156.5400,129.597896,2.0,7156.5400,0.0,572.523200,178.913500,3578.270000
4,2011-06-04,15012.1782,302.834321,5.0,15012.1782,0.0,1200.974256,375.304455,3002.435640


In [39]:
merged_data.to_csv('regression_dataset.csv', index=False)